In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [18]:
psp = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/pspnet_all.csv')
df = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/LineContainsPoints.csv')

street = gpd.read_file('/Users/hemingyi/Dropbox/New_York_SVI/Output/anglecalculation.shp')
route = pd.read_csv('/Users/hemingyi/Dropbox/New_York_SVI/route_clean.csv')


In [19]:
#merge with index to delete the route which zscore > 1.96
route = route[['index']]
df = route.merge(df, left_on='index', right_on='index', how='inner')

In [21]:
df = df[['index', 'count', 'points','route_leng']]

In [22]:
pspvalue = psp[['wall', 'building', 'tree', 'road', 'sidewalk',
       'car', 'fence', 'railing', 'signboard', 'sky', 'person', 'skyscraper',
       'van', 'awning', 'streetlight', 'ashcan', 'ceiling', 'chair', 'bicycle',
       'grass', 'earth', 'bridge', 'minibike', 'plant', 'water', 'column',
       'sculpture', 'mountain', 'booth', 'windowpane', 'pier',
       'bulletin board', 'fountain']].values
# compact the value from psp net
def val(x):
    a = int(x['Unnamed: 0'])
    return pspvalue[a]
psp['pspval'] = psp.apply(val, axis=1)

In [23]:
#get the length of lines
street['len'] = street['geometry'].length
street['FID'] = street.index
psp = psp.merge(street, left_on='FID', right_on='FID', how='inner')
psp['val'] = psp['len'] * psp['pspval']

In [24]:
# delete point which has no value
def one(x):
    return len(x['val'])
psp['one'] = psp.apply(one, axis=1)
psp = psp[psp['one'] != 0]

In [25]:
psp = psp[['FID', 'val']]

In [26]:
vals = ['wall', 'building', 'tree', 'road', 'sidewalk',
       'car', 'fence', 'railing', 'signboard', 'sky', 'person', 'skyscraper',
       'van', 'awning', 'streetlight', 'ashcan', 'ceiling', 'chair', 'bicycle',
       'grass', 'earth', 'bridge', 'minibike', 'plant', 'water', 'column',
       'sculpture', 'mountain', 'booth', 'windowpane', 'pier',
       'bulletin board', 'fountain']

In [27]:
# the column point is str
def tolist(x):
    if len(x['points']) > 3:
        point_list = [int(x) for x in x['points'][1:-1].split(',')]
        return point_list
    else:
        return np.nan

In [28]:
%%time
df['POINTS'] = df.apply(tolist, axis=1)

CPU times: user 2.58 s, sys: 153 ms, total: 2.73 s
Wall time: 2.76 s


In [29]:
psp_point = psp['FID'].values

In [30]:
# sub = df.sample(10,random_state=1)
def only_point(x):
    retA = [i for i in x['POINTS'] if i in psp_point]
    if len(retA) == 0:
        return np.nan
    else: return retA

In [31]:
%%time
# score = np.array([])
# sub = df.sample(10,random_state=1)
df['p'] = df.apply(only_point, axis=1)

CPU times: user 23 s, sys: 55.9 ms, total: 23.1 s
Wall time: 23.2 s


In [32]:
df = df.dropna()

In [33]:
psp[psp['FID'] == 3959]['val'].values[0]

array([           nan, 5.20423373e-05, 1.49995549e-08, 6.49868873e-06,
       5.78640724e-06, 2.05388642e-07, 3.43937162e-07,            nan,
                  nan,            nan, 1.04141647e-07,            nan,
       7.19715485e-08,            nan, 4.31566141e-08,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan,            nan,            nan,            nan,
                  nan])

In [34]:
df.head()

,index,count,points,route_leng,POINTS,p
0,2,13,"[22, 60, 142, 144, 396, 577, 592, 692, 967, 10...",0.041114,"[22, 60, 142, 144, 396, 577, 592, 692, 967, 10...","[22, 60, 142, 144, 577, 967, 1004, 1041, 1221,..."
1,5,4,"[14, 40, 111, 160, 219, 269, 270, 382, 722, 74...",0.052508,"[14, 40, 111, 160, 219, 269, 270, 382, 722, 74...","[40, 111, 219, 270, 879, 982, 1041, 1125, 1170..."
2,8,11,"[139, 147, 148, 219, 278, 434, 548, 562, 644, ...",0.063245,"[139, 147, 148, 219, 278, 434, 548, 562, 644, ...","[139, 219, 548, 562, 644, 844, 879, 905, 951, ..."
3,13,1,"[18, 36, 77, 129, 166, 173, 177, 245, 268, 291...",0.113139,"[18, 36, 77, 129, 166, 173, 177, 245, 268, 291...","[18, 36, 77, 129, 166, 245, 268, 291, 314, 413..."
4,14,1,"[18, 36, 77, 129, 166, 173, 177, 268, 291, 314...",0.107691,"[18, 36, 77, 129, 166, 173, 177, 268, 291, 314...","[18, 36, 77, 129, 166, 268, 291, 314, 495, 505..."


In [35]:
df = df[['index', 'count', 'route_leng', 'p']]

In [36]:
def score_cal(x):
#     matrix = []
    a = np.zeros(33)
    for i,j in enumerate(x['p']):
#         print(j)
#         a = np.r_[a, psp[psp['FID'] == j]['val'].values[0]]
        a = np.row_stack((a,psp[psp['FID'] == j]['val'].values[0]))
#         print(a)
#         locals()['var'+ str(i)] = psp[psp['FID'] == j]['val'].values
#         if i > 0:
# #                 print(j)
#             a = np.r_[a, psp[psp['FID'] == j]['val'].values]
#             locals()['var'+ str(i - 1)] = np.r_[locals()['var'+ str(i - 1)], locals()['var'+ str(i)]]
#         else: 
#             pass              
    return (np.nanmean(a[1:], axis=0))

In [37]:
%%time

df['score'] = df.apply(score_cal, axis=1)

/Users/hemingyi/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


CPU times: user 22min 38s, sys: 11.3 s, total: 22min 50s
Wall time: 23min 17s


In [38]:
df.head()

,index,count,route_leng,p,score
0,2,13,0.041114,"[22, 60, 142, 144, 577, 967, 1004, 1041, 1221,...","[0.00021746943193852886, 0.0004483728406138438..."
1,5,4,0.052508,"[40, 111, 219, 270, 879, 982, 1041, 1125, 1170...","[7.438415370421523e-05, 0.00035603138674438763..."
2,8,11,0.063245,"[139, 219, 548, 562, 644, 844, 879, 905, 951, ...","[9.115639164809638e-05, 0.0003050005168238089,..."
3,13,1,0.113139,"[18, 36, 77, 129, 166, 245, 268, 291, 314, 413...","[4.614046015685048e-05, 0.00017066230194245175..."
4,14,1,0.107691,"[18, 36, 77, 129, 166, 268, 291, 314, 495, 505...","[4.740244740068146e-05, 0.000163750074007546, ..."


In [ ]:
np.append(df['score'].values, df['', axis=1)
array([[1, 2, 3, 0],
       [2, 3, 4, 0]])

In [45]:
a = np.zeros([len(df),34])
# for i in range(len(df)):
#     a[i] = df['score'].values.tolist()[i].tolist()

In [ ]:
a.sha

In [ ]:
var = np.c_(df['score'].values

In [686]:
np.save('count',df['count'].values)

In [688]:
np.save('var',df['score'].values)